In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

path = '/home/thesby/projects/pretrained_models/LLM/Qwen1.5-MoE-A2.7B-Chat-GPTQ-Int4'
model = AutoModelForCausalLM.from_pretrained(
    path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(path)


def chat(prompt):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

/home/thesby/anaconda3/envs/qwen_2.7b_vl/lib/python3.10/site-packages/transformers/modeling_utils.py:4256: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


KeyboardInterrupt: 

In [2]:
prompt = "介绍一下你自己"
chat(prompt)

'我是来自阿里云的超大规模语言模型，我叫通义千问。作为一个AI助手，我的目标是帮助用户获得准确、有用的信息，解决他们的问题和困惑。我可以回答各种问题，提供知识性、咨询性的信息，还可以进行多轮对话，并且能够根据上下文进行推理和理解。我会不断学习和进步，不断提升自己的能力，为用户提供更好的服务。如果您有任何问题或需要帮助，请随时告诉我，我会尽力回答您的问题。'

In [26]:
prompt = '''从下列标题中提取信息，包括但不限于：品牌名、型号、功率、颜色、尺码、材质。如果没有相关信息，就置空
海尔KT039不锈钢电热水壶800W宿舍可用'''
print(chat(prompt))

根据您的描述，这款产品的品牌是海尔，型号是KT039，功率为800瓦，材质为不锈钢，适合在宿舍使用。关于颜色和尺码信息没有明确说明，所以空白。


In [8]:
tokenizer

Qwen2TokenizerFast(name_or_path='/home/thesby/projects/pretrained_models/LLM/Qwen1.5-MoE-A2.7B-Chat-GPTQ-Int4', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [13]:
tokenizer('中华人民共和国')

{'input_ids': [105492, 104773], 'attention_mask': [1, 1]}

In [17]:
tokenizer2 = AutoTokenizer.from_pretrained('/home/thesby/projects/pretrained_models/LLM/Qwen-VL-Chat-Int4', trust_remote_code=True)

In [18]:
tokenizer2

QWenTokenizer(name_or_path='/home/thesby/projects/pretrained_models/LLM/Qwen-VL-Chat-Int4', vocab_size=151860, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	
}

In [20]:
tokenizer2('中华人民共和国')

{'input_ids': [105492, 104773], 'token_type_ids': [0, 0], 'attention_mask': [1, 1]}

In [25]:
print(tokenizer.chat_template)

{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful assistant<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content']}}{% if (loop.last and add_generation_prompt) or not loop.last %}{{ '<|im_end|>' + '
'}}{% endif %}{% endfor %}{% if add_generation_prompt and messages[-1]['role'] != 'assistant' %}{{ '<|im_start|>assistant
' }}{% endif %}
